In [7]:
import csv
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import joblib

file_feature = "./csv/endometrium.csv"
file_train = "./csv/train.csv"
file_validate = "./csv/validation.csv"
file_test = "./csv/test.csv"

f = open(file_feature)
csv_f = csv.reader(f)
features = next(csv_f)
dataset = pd.read_csv(file_feature, names=features, usecols=range(1,6098), dtype=np.float64, skiprows=1, low_memory=False)
f = open(file_train)
csv_f = csv.reader(f)
features = next(csv_f)
dataset_train = pd.read_csv(file_train, names=features, usecols=range(1,1), dtype=np.float64, skiprows=1, low_memory=False)

with open('./csv/train.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    train_list = [row[1] for row in reader]
with open('./csv/validation.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    validation_list = [row['patient'] for row in reader]
with open('./csv/test.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    test_list = [row['patient'] for row in reader]

dataset['outcome'] = pd.to_numeric(dataset['outcome'],errors='coerce')
array_OG = dataset.values
print(array_OG.shape)
train_list = train_list[1:]
validation_list = validation_list[0:]
test_list = test_list[0:]
#print(test_list)
#print(train_list)
#print(validation_list)

def cat_str(num_list):
    n_list = []
    for i in num_list:
        temp = i[12:]
        n_list.append(temp)
    n_list = [int(x) for x in n_list]
    return n_list

train_list = cat_str(train_list)
validation_list = cat_str(validation_list)
test_list = cat_str(test_list)

#print(train_list)
#print(validation_list)
#print(test_list)
#print(len(test_list))

CT = [1,2,7,12,16,17,29,30,31,33,34,35,37,38,40,42,45,51,54,56,57,58,59,60,62,67,69,79,84,109,110,116,119,137,138,143,404,147,155,156,161,201,232,248,256,269,278,624,635,646,650,358,366,379,393,395,561,689,693,702,737,778,789,805]
        
new_train = []
new_validation = []
new_test = []
for i in CT:
    if i in test_list:
        new_test.append(i)
    if i in validation_list:
        new_validation.append(i)
    if i in train_list:
        new_train.append(i)

print(len(new_train))
print(len(new_validation))
print(len(new_test))

train_list = new_train
validation_list = new_validation
test_list = new_test
        
'''
train_list:
1
12
29
30
31
35
37
38
42
45
51
54
57
58
60
67
79
84
109
110
116
119
137
138
143
404
147
155
156
161
201
232
248
256
269
278
646
650
358
366
379
393
395
561
689
693
778
789
'''

'''
validation_list
7
16
33
56
59
69
702
737
'''

'''
test_list
2
17
34
40
62
624
635
805
'''

(849, 6097)
48
8
8


'\ntest_list\n2\n17\n34\n40\n62\n624\n635\n805\n'

In [11]:
train_feature = []
validate_feature = []
test_feature = []
count = 1
for i in range(len(array_OG)):
    num = i + 1
    if num in train_list:
        train_feature.append(array_OG[i])
    elif num in validation_list:
        validate_feature.append(array_OG[i])
    elif num in test_list:
        #print(count)
        count = count + 1
        test_feature.append(array_OG[i])
        
train_feature = np.array(train_feature)
validate_feature = np.array(validate_feature)
test_feature = np.array(test_feature)

train_feature = pd.DataFrame(train_feature)
train_feature.dropna(axis=1, thresh=2, inplace=True)
#train_feature.dropna(how='all',thresh = 20,inplace=True)
train_feature = np.array(train_feature)
wh_inf = np.isinf(train_feature)
train_feature[wh_inf]=0
wh_nan = np.isnan(train_feature)
train_feature[wh_nan]=0

validate_feature = pd.DataFrame(validate_feature)
validate_feature.dropna(axis=1, thresh=2, inplace=True)
#validate_feature.dropna(how='all',thresh = 20,inplace=True)
validate_feature = np.array(validate_feature)
wh_inf = np.isinf(validate_feature)
validate_feature[wh_inf]=0
wh_nan = np.isnan(validate_feature)
validate_feature[wh_nan]=0

test_feature = pd.DataFrame(test_feature)
test_feature.dropna(axis=1, thresh=2, inplace=True)
#test_feature.dropna(how='all',thresh = 20,inplace=True)
test_feature = np.array(test_feature)
wh_inf = np.isinf(test_feature)
test_feature[wh_inf]=0
wh_nan = np.isnan(test_feature)
test_feature[wh_nan]=0

#only use image features
X_train = train_feature[:,:6093]
Y_train = train_feature[:,6093]
Y_train = Y_train.astype('int32')

X_validate = validate_feature[:,:6093]
Y_validate = validate_feature[:,6093]
Y_validate = Y_validate.astype('int32')

X_test = test_feature[:,:6093]
Y_test = test_feature[:,6093]
Y_test = Y_test.astype('int32')


print(len(Y_train))
print(len(Y_validate))
print(len(Y_test))

48
8
8


In [23]:
pipe = joblib.load('./handpkl/EndoBAGRELF20.pkl')
Y_pred = pipe.predict(X_validate)
print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
print("Specificity" + repr(tn/(tn+fp)))
print(tn,fp,fn,tp)

Accuracy: 0.875
Average Precision Score: 0.9583333333333334
Kappa: 0.7142857142857143
Hamming Loss: 0.125
AUC0.9166666666666667
Sensitivity0.8333333333333334
Specificity1.0
2 0 1 5


In [25]:
for i in Y_validate:
    print(i)

1
1
1
1
1
1
0
0


In [14]:
Y_pred = pipe.predict(X_test)
print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
print("Specificity" + repr(tn/(tn+fp)))
print(tn,fp,fn,tp)

Accuracy: 0.75
Average Precision Score: 0.9642857142857143
Kappa: 0.3846153846153846
Hamming Loss: 0.25
Sensitivity0.7142857142857143
Specificity1.0
1 0 2 5


In [27]:
for i in Y_test:
    print(i)

1
1
1
1
1
1
1
0


In [20]:
Y_pred = pipe.predict(X_train)
print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
print("Specificity: " + repr(tn / (tn + fp)))

Accuracy: 0.8958333333333334
Average Precision Score: 0.9489931107578167
Kappa: 0.7309417040358744
Hamming Loss: 0.10416666666666667
AUC: 0.9004914004914005
Sensitivity: 0.8918918918918919
Specificity: 0.9090909090909091


In [22]:
for i in Y_train:
    print(i)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
1
1
0
0
0
0
